In [16]:
import pandas as pd

colunas = [
    'Coluna_1',
    'Coluna_2',
    'Coluna_3',
    'Coluna_4'
]

datatype={'Coluna_1':'float','Coluna_2':'float','Coluna_3':'float','Coluna_4':'float'}

df = pd.DataFrame(columns=colunas,dtype=datatype)
df.info()

ValueError: entry not a 2- or 3- tuple

In [11]:
datatype={'Coluna_1':'str','Coluna_2':'str','Coluna_3':'str','Coluna_4':'str'}

In [12]:
datatype

{'Coluna_1': 'str', 'Coluna_2': 'str', 'Coluna_3': 'str', 'Coluna_4': 'str'}

In [ ]:
def register_source_files(source_files: list):
    for source_file in source_files:
        headerExists = 'true' #source_file['headerExists'].lower().strip() == 'true'
        
        if headerExists:
            #schema = None
            schema = parse_schema(source_file['header'])
            
        else:
            schema = parse_schema(source_file['header'])
        
        df = spark.read.option("maxColumns", 100000000).csv(
            source_file['folderName'] + "/" + source_file['fileName'] + "." + source_file['fileExtension'],
            sep=source_file['delimiter'],
            header=headerExists,
            schema=schema,
           encoding="ISO-8859-1"
        )
        df.createOrReplaceTempView(source_file['name'])

In [ ]:
def get_update_set(update_columns: str) -> str:
    fields_name = [field.strip() for field in update_columns.split(',')]
    update_set_list = ['{0}.{1} = {0}_delta.{1}'.format(targetTableName, field) for field in fields_name]
    update_set = ', \n'.join(update_set_list)
    
    return update_set

In [ ]:
configs = read_json('/dbfs/mnt/swamp01/anp/anp_dadosabertos.json')
source_files = configs['sourceFiles']
register_source_files(source_files)

In [ ]:
deltaTableLocation = configs['deltaTableLocation']
targetTableName = configs['targetTableName']
targetTableSchema = configs['targetTableSchema']
targetFields_list = [field.strip().split(' ')[0] for field in targetTableSchema.split(',')]
targetFields = ','.join(targetFields_list)
tableLocation = configs['tableLocation']
mergeKeys = configs['mergeKeys']
mergeKeys_list = [field.strip().split(' ')[0] for field in mergeKeys.split(',')]
updateColumns = configs['updateColumns']
update_set = get_update_set(updateColumns)